In [1]:
from collections import Counter
from pymystem3 import Mystem
from itertools import chain
import re
from tqdm import tqdm
from random import shuffle
import numpy as np
from scipy.optimize import minimize
from permute.core import two_sample

# Lemmatization

Очистим данные и лемматизируем слова

In [2]:
def get_words(filename):
    with open(filename) as fp:
        lines = [re.sub("[^а-я\s]", "", line.lower()) for line in fp.readlines()]
    lines = [re.sub("\s+", " ", line).strip() for line in lines]
    stemmer = Mystem()
    words = chain.from_iterable([
        [
            word for word in stemmer.lemmatize(line) if re.match('[а-я]+', word)
        ] 
        for line in tqdm(lines)
    ]
    )
    words = list(words)
    shuffle(words)
    return words

In [3]:
tolstoy_words = get_words('tolstoj_lew_nikolaewich-text_0080.txt')
news_words = get_words('ru.txt')

100%|██████████| 1997/1997 [00:03<00:00, 610.42it/s]


In [4]:
tolstoy_count = Counter(tolstoy_words)
news_count = Counter(news_words)

print(f"Number of words in Anna Karenina: {len(tolstoy_words)}")
print(f"Number of words in news corpus: {len(news_words)}" + "\n")
print(f"Number of unique words in Anna Karenina: {len(tolstoy_count)}")
print(f"Number of unique words in news corpus: {len(news_count)}")

Number of words in Anna Karenina: 268392
Number of words in news corpus: 39020

Number of unique words in Anna Karenina: 12396
Number of unique words in news corpus: 6359


In [5]:
tolstoy_count.most_common(10)

[('и', 12905),
 ('он', 9298),
 ('не', 6534),
 ('что', 6063),
 ('она', 6001),
 ('в', 5715),
 ('быть', 5282),
 ('я', 4491),
 ('на', 3594),
 ('с', 3328)]

In [6]:
news_count.most_common(10)

[('в', 1964),
 ('и', 1013),
 ('что', 759),
 ('на', 707),
 ('быть', 614),
 ('он', 428),
 ('с', 401),
 ('который', 378),
 ('не', 371),
 ('как', 343)]

# MLE of Zipf's law distribution

Zipf's law:  $$\mathbb{P}\left[rank(x)=k\right|a] = \frac{c}{k^a}$$
where $x$ represents a word, $N$ - total number of words, $rank(x)$ - rank of the word determined by sample.

C is determined by normalizing conditions:
$$c = \frac{1}{\sum\limits_{k=1}^{N}\frac{1}{k^a}}$$
Likelihood for a sample $x = (x_1, x_2, ..., x_n)$ of size n:
$$L = c^n\left(\prod\limits_{i=1}^{n}\frac{1}{(rank(x_i))^a}\right)$$
$$\ln(L) = -n\ln\left(\sum\limits_{k=1}^{N}\frac{1}{k^a}\right) - a\sum\limits_{i=1}^{n}\ln(rank(x_i))$$

$$\frac{1}{n}\ln(L) = -\ln\left(\sum\limits_{k=1}^{N}\frac{1}{k^a}\right) - a\frac{1}{n}\sum\limits_{i=1}^{n}\ln(rank(x_i))$$

In [7]:
def logL(a, sample):
    N = max(sample)
    c = 1.0/np.sum([(1.0/k)**a for k in range(1, N+1)])
    lnL = np.log(c) - a*np.mean(np.log(sample))
    return lnL

In [11]:
# Maximizing loglikelihood is equivalent to minimizing -loglikelihood
def find_a(sample):
    return minimize(lambda a: -logL(a, sample), x0=[0.9]).x[0]

In [9]:
# Переведем выборку из слов в ранги 
tolstoy_rank = {w:i+1 for i, (w, c) in enumerate(tolstoy_count.most_common())}
news_rank = {w:i+1 for i, (w, c) in enumerate(news_count.most_common())}
tolstoy_s = [tolstoy_rank[w] for w in tolstoy_words]
news_s = [news_rank[w] for w in news_words]

In [12]:
tolstoy_a = find_a(tolstoy_s)

In [13]:
news_a = find_a(news_s)

In [14]:
print(f"Parameter 'a' for Anna Karenina:{tolstoy_a}")
print(f"Parameter 'a' for news corpus:{news_a}" + "\n")

tolstoy_c = 1.0/np.sum([(1.0/k)**tolstoy_a for k in range(1, len(tolstoy_rank)+1)])
news_c = 1.0/np.sum([(1.0/k)**news_a for k in range(1, len(news_rank)+1)])

print(f"Parameter 'c' for Anna Karenina:{tolstoy_c}")
print(f"Parameter 'c' for news corpus:{news_c}")

Parameter 'a' for Anna Karenina:1.003967666194625
Parameter 'a' for news corpus:0.8738136100564119

Parameter 'c' for Anna Karenina:0.10174369488436825
Parameter 'c' for news corpus:0.06034151087263045


# Chi2 test

Проверим, дейстительно ли мы вероятности подчиняются закону Ципфа

In [15]:
import scipy.stats as st

In [16]:
# Observed frequencies
tolstoy_obs_f = np.array([tolstoy_count[w] for w in tolstoy_rank]) 
tolstoy_obs_f = tolstoy_obs_f/np.sum(tolstoy_obs_f)
# Calculated Zipf's law frequencies
tolstoy_exp_f = [tolstoy_c/k**tolstoy_a for k in range(1, len(tolstoy_rank)+1)] 
st.chisquare(f_obs=tolstoy_obs_f, f_exp=tolstoy_exp_f, ddof=1)

Power_divergenceResult(statistic=0.11684582924175284, pvalue=1.0)

In [17]:
news_obs_f = np.array([tolstoy_count[w] for w in news_rank])
news_obs_f = news_obs_f/np.sum(news_obs_f)
news_exp_f = [tolstoy_c/k**news_a for k in range(1, len(news_rank)+1)]
st.chisquare(f_obs=news_obs_f, f_exp=news_exp_f, ddof=1)

Power_divergenceResult(statistic=2.0895418051454455, pvalue=1.0)

Как видим, не можем отвергнуть нулевую гипотезу о том, что распределение подчинено закону Ципфа (pvalue=1.0).

Будем считать предположение верным.

# Test for equality

Получим доверительные интервалы для нашей оценки $a_{MLE}$ для обоих выборок с помощью бутстрепа.

In [18]:
from arch.bootstrap import IIDBootstrap

In [19]:
%%time
bs = IIDBootstrap(np.array(tolstoy_s))
print(bs.conf_int(find_a, 1000, size=0.95, method='basic'))

[[1.00292161]
 [1.00506477]]
CPU times: user 13min 44s, sys: 10.2 s, total: 13min 54s
Wall time: 14min 1s


In [20]:
%%time
bs = IIDBootstrap(np.array(news_s))
print(bs.conf_int(find_a, 1000, size=0.95, method='basic'))

[[0.87006576]
 [0.87739855]]
CPU times: user 4min 10s, sys: 4.52 s, total: 4min 14s
Wall time: 4min 17s


Как видим, 95% доверительные интервалы оценок не пересекаются, и мы можем отвергнуть гипотезу о равенстве параметров распределений.